In [1]:
from gurobipy import Model, GRB, quicksum, tuplelist
import numpy as np
import pandas as pd

In [2]:
# The following file works as a reference table to match the names of 2 sheets
hub_list = pd.read_csv('Database/Hub list.csv')
hub_list.Ref_City = hub_list.Ref_City + "-" + hub_list.Country

# drop duplicates
hub_list.drop_duplicates(subset = "Ref_City", keep='first', inplace=True)


hub_ref = {}
for row in hub_list.index:
    hub_ref[hub_list.loc[row, "City"]] = hub_list.loc[row, "Ref_City"]

    
    
def name_transform(city_name):
    if city_name in hub_ref.keys():
        return hub_ref[city_name]
    else:
        return city_name
    
    



# Data Preparation

## 1. Used Routes:

In [3]:
used_routes = pd.read_csv("Output/freq_output.csv")
used_routes.columns = ["", "Origin", "Destination", "Truck Type", "Freq"]
used_routes = used_routes[used_routes["Freq"] > 0]

used_routes_gb = used_routes.groupby(["Origin", "Destination"]).count()["Truck Type"].reset_index()

route_list = list(zip(used_routes_gb.Origin.tolist(), used_routes_gb.Destination.tolist()))
used_hubs = used_routes_gb.Origin.tolist() + used_routes_gb.Destination.tolist()
used_hubs = list( dict.fromkeys(used_hubs))

In [4]:
hub_list_used = hub_list.set_index("Ref_City")
hub_list_used = hub_list_used.loc[hub_list_used.index.isin(used_hubs), :]
hub_list_used.reset_index(inplace = True)


## 2. Travel time (TT)
Total transportation time from hub p ∈ H to q ∈ H

In [5]:
tt_file = pd.read_csv("Database/travel time.csv")
TT = {}
for row in tt_file.index:
    key = (tt_file.loc[row, "From"], tt_file.loc[row, "To"])
    value = tt_file.loc[row, "est_tt"]
    
    TT[key] = value

## 3. Hub Open & Cut-off time
- Op = Hub opened time for hub p ∈ H
- COp  = Hub cut off time for hub p ∈ H
- Close  = Hub close time for hub p ∈ H

In [6]:
hub_info = pd.read_csv("Database/hub_info.csv")
merged_hub_info = hub_list_used.merge(hub_info[["Hub", "Open", "Cut-off", "Close"]], left_on = "Ref_City", right_on = "Hub", how = "left")

merged_hub_info["Open"].fillna(merged_hub_info["Open"].mode().values[0], inplace = True)
merged_hub_info["Close"].fillna(merged_hub_info["Close"].mode().values[0], inplace = True)
merged_hub_info["Cut-off"].fillna(merged_hub_info["Cut-off"].mode().values[0], inplace = True)
# to replace certain mistakes:
# merged_hub_info["Cut-off"] = merged_hub_info.apply(lambda row: row["Cut-off"] if row["Cut-off"] < 10 else row["Cut-off"], axis = 1)

O = {} # Open time
for row in merged_hub_info.index:
    key = merged_hub_info.loc[row, "Ref_City"]
    value = merged_hub_info.loc[row, "Open"]
    O[key] = value

CO = {} # Cut-off time
for row in merged_hub_info.index:
    key = merged_hub_info.loc[row, "Ref_City"]
    value = merged_hub_info.loc[row, "Cut-off"]
    CO[key] =  value
    
Close = {} # Close time
for row in merged_hub_info.index:
    key = merged_hub_info.loc[row, "Ref_City"]
    value = merged_hub_info.loc[row, "Close"]
    Close[key] =  value


# Model

In [7]:
model = Model("DHL")

Using license file /Users/yi/gurobi.lic
Academic license - for non-commercial use only


In [8]:
S = model.addVars(route_list , vtype=GRB.CONTINUOUS, name='Spq')         # Start time from hub p

N = model.addVars(route_list , vtype=GRB.INTEGER, name='Npq')         # Night Needed from hub p to hub q

J = model.addVars(route_list , vtype=GRB.CONTINUOUS, name='Npq') # Penalty when it can not arrive the hub on time


In [9]:
model.update()

# Minimize the travel time for each route
model.setObjective(quicksum(J[p,q] for p, q in route_list),  GRB.MINIMIZE)


In [10]:
model.addConstrs((S[p,q] + TT[p,q] - N[p,q] * 24 + J[p,q]  <=  
                 CO[q] for p,q in route_list), "Hub Cut Off Time")

model.addConstrs((S[p,q] + TT[p,q] - N[p,q] * 24 + J[p,q] >=  
                 O[q] for p,q in route_list), "Hub Open Time")

model.addConstrs((S[p,q]<=  
                 Close[p] for p,q in route_list), "Hub Close Time for departure")
model.addConstrs((S[p,q] >=  
                 O[p] for p,q in route_list), "Hub Open Time for departure")


{('Araba (Vitoria)-ES', 'Brüssel-BE'): <gurobi.Constr *Awaiting Model Update*>,
 ('Araba (Vitoria)-ES',
  'Eindhoven-Noord-NL'): <gurobi.Constr *Awaiting Model Update*>,
 ('Araba (Vitoria)-ES', 'Lisboa-PT'): <gurobi.Constr *Awaiting Model Update*>,
 ('Araba (Vitoria)-ES', 'Slough-GB'): <gurobi.Constr *Awaiting Model Update*>,
 ('Barcelona-ES', 'Helsingborg-SE'): <gurobi.Constr *Awaiting Model Update*>,
 ('Barcelona-ES', 'Hub Wien I-AT'): <gurobi.Constr *Awaiting Model Update*>,
 ('Barcelona-ES', 'PZ 77 (Lahr)-DE'): <gurobi.Constr *Awaiting Model Update*>,
 ('Bettembourg-LU', 'Ljubljana-SI'): <gurobi.Constr *Awaiting Model Update*>,
 ('Brno-CZ', 'Budapest OLK-HU'): <gurobi.Constr *Awaiting Model Update*>,
 ('Brno-CZ', 'Hagenbrunn-AT'): <gurobi.Constr *Awaiting Model Update*>,
 ('Brno-CZ', 'Ivanka pri Dunaji-SK'): <gurobi.Constr *Awaiting Model Update*>,
 ('Brüssel-BE', 'Araba (Vitoria)-ES'): <gurobi.Constr *Awaiting Model Update*>,
 ('Brüssel-BE', 'Combs la ville-FR'): <gurobi.Constr *A

In [11]:
model.update()
model.printStats()


Statistics for model DHL :
  Linear constraint matrix    : 636 Constrs, 477 Vars, 1272 NZs
  Variable types              : 318 Continuous, 159 Integer
  Matrix coefficient range    : [ 1, 24 ]
  Objective coefficient range : [ 1, 1 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 0.00333333, 56.57 ]


In [12]:
timeLimit = 300
model.Params.TimeLimit = timeLimit - model.getAttr(GRB.Attr.Runtime)
model.optimize()

Changed value of parameter TimeLimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 636 rows, 477 columns and 1272 nonzeros
Model fingerprint: 0xc7ee97bf
Variable types: 318 continuous, 159 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-03, 6e+01]
Found heuristic solution: objective 482.0133333
Presolve removed 634 rows and 474 columns
Presolve time: 0.06s
Presolved: 2 rows, 3 columns, 6 nonzeros
Found heuristic solution: objective 13.8033333
Variable types: 2 continuous, 1 integer (0 binary)

Root relaxation: objective 4.933333e-01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.4933333    0.49333  0.00%     -    0s

In [13]:
def turn_to_df(Var, col_name = None):  # Here you could input the variable
    solution = model.getAttr('x_', Var)
    num_col = len(solution.keys()[0])
    collect = []
    for row in range(0, len(solution)):
        new_row = []
        for col in range(0, num_col):
            new_row.append(solution.keys()[row][col])
        new_row.append(solution[solution.keys()[row]])
    
        collect.append(new_row)
    return pd.DataFrame(collect, columns = col_name)



In [14]:
Start = turn_to_df(S, ["Ori_Hub", "Dest_Hub", "Time"])
Night = turn_to_df(N, ["Ori_Hub", "Dest_Hub", "Nights"])
Penalty = turn_to_df(J, ["Ori_Hub", "Dest_Hub", "Extra Hour"])



In [15]:
attri_name = ["start", "night", "penalty"]
all_result = [Start, Night, Penalty]

for row in range(0, len(attri_name)):
    all_result[row].to_csv(f"Output/{attri_name[row]}_output.csv")

In [16]:
Penalty[Penalty["Extra Hour"] > 0]

,Ori_Hub,Dest_Hub,Extra Hour
45,Göteborg-SE,Oslo-NO,0.493333
